In [1]:
pip install seaborn

This notebook pulls data from the [Emissions Impact Cumulative](https://docs.google.com/spreadsheets/d/1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE/edit?gid=1466607796#gid=1466607796) sheet of the Tokenomics model.

  
1.   EDA: we pull data and explore utilization changes
2.   Revenue driven by emissions: we calc revenue from vaults that were provided emissions in the last several epochs (90 days)
3.   Annualized revenue driven by emissions

## EDA

In [2]:
# @title
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import auth
import gspread
from google.auth import default
from oauth2client.client import GoogleCredentials
import plotly.express as px
import plotly.graph_objects as go

def authenticate_and_get_data():
    auth.authenticate_user()
    credentials, _ = default()
    gc = gspread.authorize(credentials)

    sheet_id = '1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE'
    worksheet_id = '1466607796'

    workbook = gc.open_by_key(sheet_id)
    worksheet = workbook.get_worksheet_by_id(int(worksheet_id))

    data = worksheet.get_all_records()
    df = pd.DataFrame(data)
    return df

# Load and clean data
df = authenticate_and_get_data()

def extract_network(vault_name):
    if 'm' in vault_name.lower():
        return 'Mode'
    elif 'b' in vault_name.lower():
        return 'Base'
    elif 'o' in vault_name.lower():
        return 'Optimism'
    else:
        return 'Unknown'

# Add network column
df['network'] = df['vaultName'].apply(extract_network)

numeric_cols = ['Total Supply at Start', 'Total Borrowed at Start',
                'Total Supply at End', 'Total Borrowed at End',
                'Change in Supply %', 'Change in Borrow %']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    # Replace infinite values with NaN
    df[col] = df[col].replace([np.inf, -np.inf], np.nan)

# calc utilization rates with error handling
df['Initial Utilization'] = np.where(
    df['Total Supply at Start'] > 0,
    (df['Total Borrowed at Start'] / df['Total Supply at Start'] * 100),
    0
).round(2)

df['Final Utilization'] = np.where(
    df['Total Supply at End'] > 0,
    (df['Total Borrowed at End'] / df['Total Supply at End'] * 100),
    0
).round(2)

df['Utilization Change'] = (df['Final Utilization'] - df['Initial Utilization']).round(2)

# Fill NaN values with 0 for plotting
df_plot = df.copy()
df_plot['Final Utilization'] = df_plot['Final Utilization'].fillna(0)

#  multiple interactive plots
def create_interactive_plots(df_plot):
    # Supply vs Borrow Changes
    fig1 = px.scatter(df_plot,
                     x='Change in Supply %',
                     y='Change in Borrow %',
                     color='network',
                     size=np.abs(df_plot['Final Utilization']) + 1,  # Add 1 to avoid zero sizes
                     hover_data=['vaultName'],
                     title='Supply vs Borrow Changes by Network')

    # Network Performance Box Plot
    fig2 = go.Figure()
    networks = df_plot['network'].unique()

    for metric in ['Change in Supply %', 'Change in Borrow %']:
        for network in networks:
            fig2.add_trace(go.Box(
                y=df_plot[df_plot['network'] == network][metric],
                name=network,
                boxpoints='all',
                jitter=0.3,
                pointpos=-1.8,
                legendgroup=metric,
                showlegend=True,
                text=df_plot[df_plot['network'] == network]['vaultName']
            ))

    fig2.update_layout(
        title='Distribution of Changes by Network',
        yaxis_title='Percentage Change',
        boxmode='group'
    )

    # Utilization Changes
    fig3 = px.bar(df_plot.sort_values('Utilization Change'),
                  x='vaultName',
                  y='Utilization Change',
                  color='network',
                  title='Utilization Change by Vault')

    fig3.update_layout(
        xaxis_tickangle=-45,
        xaxis_title='Vault Name',
        yaxis_title='Change in Utilization (%)'
    )

    return fig1, fig2, fig3

# Print statistics
print("=== Network Summary ===")
print(network_summary(df))
print("\n=== Top Performers by Supply Growth ===")
print(top_performers(df, 'Change in Supply %'))
print("\n=== Top Performers by Borrow Growth ===")
print(top_performers(df, 'Change in Borrow %'))

#  display  plots
fig1, fig2, fig3 = create_interactive_plots(df_plot)
fig1.show()
fig2.show()
fig3.show()

# additional analysis
print("\n=== Key Statistics by Network ===")
print("\nMedian Supply Change %:")
print(df.groupby('network')['Change in Supply %'].median())
print("\nMedian Borrow Change %:")
print(df.groupby('network')['Change in Borrow %'].median())

#  summary table for top performing vaults
summary_table = pd.DataFrame({
    'Metric': ['Highest Supply Growth', 'Highest Borrow Growth', 'Most Improved Utilization'],
    'Vault': [
        df.loc[df['Change in Supply %'].idxmax(), 'vaultName'],
        df.loc[df['Change in Borrow %'].idxmax(), 'vaultName'],
        df.loc[df['Utilization Change'].idxmax(), 'vaultName']
    ],
    'Network': [
        df.loc[df['Change in Supply %'].idxmax(), 'network'],
        df.loc[df['Change in Borrow %'].idxmax(), 'network'],
        df.loc[df['Utilization Change'].idxmax(), 'network']
    ],
    'Value': [
        f"{df['Change in Supply %'].max():.2f}%",
        f"{df['Change in Borrow %'].max():.2f}%",
        f"{df['Utilization Change'].max():.2f}%"
    ]
})

print("\n=== Outstanding Performers ===")
print(summary_table.to_string(index=False))



=== Network Summary ===


NameError: name 'network_summary' is not defined

## Revenue Generated by Vault

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import auth
import gspread
from google.auth import default

def authenticate_and_get_data():
    auth.authenticate_user()
    credentials, _ = default()
    gc = gspread.authorize(credentials)
    sheet_id = '1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE'
    worksheet_id = '1466607796'
    workbook = gc.open_by_key(sheet_id)
    worksheet = workbook.get_worksheet_by_id(int(worksheet_id))
    data = worksheet.get_all_records()
    return pd.DataFrame(data)

def extract_network(vault_name):
    if 'm' in vault_name.lower():
        return 'Mode'
    elif 'b' in vault_name.lower():
        return 'Base'
    elif 'o' in vault_name.lower():
        return 'Optimism'
    else:
        return 'Unknown'

def calculate_borrow_rate(utilization):
    base_rate = 0
    multiplier = 0.18
    jump_multiplier = 4.0
    kink = 0.8
    if utilization <= kink:
        return base_rate + (utilization * multiplier)
    else:
        normal_rate = base_rate + (kink * multiplier)
        excess_util = utilization - kink
        return normal_rate + (excess_util * jump_multiplier)

def analyze_vault_revenue(df, days_elapsed=90):
    results = df.copy()

    results['Initial_Utilization'] = np.where(
        results['Total Supply at Start'] > 0,
        results['Total Borrowed at Start'] / results['Total Supply at Start'],
        0
    )

    results['Final_Utilization'] = np.where(
        results['Total Supply at End'] > 0,
        results['Total Borrowed at End'] / results['Total Supply at End'],
        0
    )

    results['Avg_Borrow_Amount'] = np.sqrt(
        results['Total Borrowed at Start'] * results['Total Borrowed at End']
    ).fillna(0)

    results['Avg_Utilization'] = np.sqrt(
        results['Initial_Utilization'] * results['Final_Utilization']
    ).fillna(0)

    results['Borrow_Rate'] = results['Avg_Utilization'].apply(calculate_borrow_rate)
    daily_interest = results['Avg_Borrow_Amount'] * results['Borrow_Rate'] / 365

    results['Period_Interest'] = daily_interest * days_elapsed
    results['Period_Revenue'] = results['Period_Interest'] * 0.10

    results['Annual_Interest'] = daily_interest * 365
    results['Annual_Revenue'] = results['Annual_Interest'] * 0.10

    results['Effective_Borrow_APR'] = results['Borrow_Rate'] * 100
    results['Supply_APR'] = results['Borrow_Rate'] * results['Avg_Utilization'] * 0.9 * 100

    return results

def create_visualizations(revenue_df, network_colors):
    # 1. Revenue Distribution Treemap
    fig1 = px.treemap(
        revenue_df[revenue_df['Period_Revenue'] > 0],
        path=[px.Constant("All Networks"), 'network', 'vaultName'],
        values='Period_Revenue',
        color='Effective_Borrow_APR',
        color_continuous_scale='Viridis',
        title='Protocol Revenue Distribution',
        hover_data=['Avg_Utilization', 'Supply_APR']
    )
    fig1.show()

    # 2. Revenue vs Utilization
    fig2 = px.scatter(
        revenue_df,
        x='Avg_Utilization',
        y='Period_Revenue',
        color='network',
        color_discrete_map=network_colors,
        title='Revenue vs Utilization',
        hover_data=['vaultName'],
        labels={'Avg_Utilization': 'Average Utilization',
                'Period_Revenue': 'Period Revenue ($)'}
    )
    fig2.update_layout(
        xaxis_title="Average Utilization (%)",
        yaxis_title="Period Revenue ($)"
    )
    fig2.show()

    # 3. APR Distribution
    fig3 = px.violin(
        revenue_df,
        x='network',
        y='Effective_Borrow_APR',
        color='network',
        color_discrete_map=network_colors,
        box=True,
        title='APR Distribution by Network',
        labels={'Effective_Borrow_APR': 'Effective Borrow APR (%)',
                'network': 'Network'}
    )
    fig3.show()

    # 4. Network Revenue Share
    fig4 = px.pie(
        values=revenue_df.groupby('network')['Period_Revenue'].sum(),
        names=revenue_df.groupby('network')['Period_Revenue'].sum().index,
        title='Network Revenue Share',
        color=revenue_df.groupby('network')['Period_Revenue'].sum().index,
        color_discrete_map=network_colors
    )
    fig4.show()

def main():
    # Load and prepare data
    df = authenticate_and_get_data()
    df['network'] = df['vaultName'].apply(extract_network)

    numeric_cols = ['Total Supply at Start', 'Total Borrowed at Start',
                   'Total Supply at End', 'Total Borrowed at End',
                   'Change in Supply %', 'Change in Borrow %']

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)

    days_elapsed = 90
    revenue_df = analyze_vault_revenue(df, days_elapsed)

    network_colors = {'Mode': '#1f77b4', 'Base': '#2ca02c', 'Optimism': '#ff7f0e'}

    # Print analysis
    print("\n=== Ionic Protocol Revenue Analysis ===")
    print(f"\nMeasurement Period: {days_elapsed} days")
    print(f"\nTotal Protocol Revenue: ${revenue_df['Period_Revenue'].sum():,.2f}")
    print(f"Annualized Revenue (Run-Rate): ${revenue_df['Annual_Revenue'].sum():,.2f}")
    print(f"Total Vaults: {len(revenue_df)}")
    print(f"Revenue-Generating Vaults: {len(revenue_df[revenue_df['Period_Revenue'] > 0])}")

    print("\nRevenue by Network:")
    for network in revenue_df['network'].unique():
        network_data = revenue_df[revenue_df['network'] == network]
        print(f"\n{network}:")
        print(f"Period Revenue: ${network_data['Period_Revenue'].sum():,.2f}")
        print(f"Annualized Revenue: ${network_data['Annual_Revenue'].sum():,.2f}")
        print(f"Avg Utilization: {(network_data['Avg_Utilization'].mean() * 100):.1f}%")
        print(f"Avg Borrow APR: {network_data['Effective_Borrow_APR'].mean():.1f}%")
        print(f"Active/Total Vaults: {len(network_data[network_data['Period_Revenue'] > 0])}/{len(network_data)}")

    # Create visualizations
    create_visualizations(revenue_df, network_colors)

    return revenue_df

if __name__ == "__main__":
    revenue_df = main()


=== Ionic Protocol Revenue Analysis ===

Measurement Period: 90 days

Total Protocol Revenue: $37,290.86
Annualized Revenue (Run-Rate): $151,235.17
Total Vaults: 49
Revenue-Generating Vaults: 32

Revenue by Network:

Base:
Period Revenue: $6,780.04
Annualized Revenue: $27,496.81
Avg Utilization: 19.1%
Avg Borrow APR: 3.7%
Active/Total Vaults: 9/21

Optimism:
Period Revenue: $530.33
Annualized Revenue: $2,150.78
Avg Utilization: 15.9%
Avg Borrow APR: 2.9%
Active/Total Vaults: 7/9

Mode:
Period Revenue: $29,980.50
Annualized Revenue: $121,587.57
Avg Utilization: 25.1%
Avg Borrow APR: 4.7%
Active/Total Vaults: 16/19
